In [1]:
from arcgis.gis import GIS
gis = GIS("home")

In [5]:
import requests
import os
import zipfile
import arcpy

In [ ]:
#set important file paths
gdb_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb"
proj_folder = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj"



In [7]:
#set urls for download

streets_url = "https://hub.arcgis.com/api/v3/datasets/2c68a692df4b4e47af378a00452d85b0_0/downloads/data?format=shp&spatialRefId=26915&where=1=1"
bike_url = "https://hub.arcgis.com/api/v3/datasets/938c8598f0a0439a8d612bb7894a05cc_1/downloads/data?format=shp&spatialRefId=26915&where=1=1"

streets_zip_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Hennepin_County_Street_Centerlines.zip"
bike_zip_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Hennepin_County_Bike_and_Pedestrian_System.zip"

streets_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Hennepin_County_Street_Centerlines"
bike_path = r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\Hennepin_County_Bike_and_Pedestrian_System"

#request downloads
streets_res = requests.get(streets_url)
bike_res = requests.get(bike_url)

#save to file
with open (streets_zip_path, "wb") as f:
    f.write(streets_res.content)
    
with open (bike_zip_path, "wb") as f:
    f.write(bike_res.content)

In [9]:
#unzip files
with zipfile.ZipFile(streets_zip_path, "r") as zip_ref:
    zip_ref.extractall(streets_path)
    
with zipfile.ZipFile(bike_zip_path, "r") as zip_ref:
    zip_ref.extractall(bike_path)

In [ ]:
## insert a few steps here I did by hand the first time

In [10]:
#narrow bike and ped system to just bike system

arcpy.conversion.ExportFeatures(
    in_features="Hennepin_County_Bike_and_Pedestrian_System",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Hennepin_Bike_Sys",
    where_clause="FACILITY = 'Bike' Or FACILITY = 'Shared'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='OBJECTID "OBJECTID" true true false 4 Short 0 4,First,#,Hennepin_County_Bike_and_Pedestrian_System,OBJECTID,-1,-1;CITY_MUNI "CITY_MUNI" true true false 33 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,CITY_MUNI,0,32;COUNTY "COUNTY" true true false 8 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,COUNTY,0,7;LENGTH "LENGTH" true true false 24 Double 15 23,First,#,Hennepin_County_Bike_and_Pedestrian_System,LENGTH,-1,-1;WIDTH "WIDTH" true true false 24 Double 15 23,First,#,Hennepin_County_Bike_and_Pedestrian_System,WIDTH,-1,-1;PARKING "PARKING" true true false 3 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,PARKING,0,2;UNIQUE_ID "UNIQUE_ID" true true false 13 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,UNIQUE_ID,0,12;TRAILNAME "TRAILNAME" true true false 53 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,TRAILNAME,0,52;TRLSYSNAME "TRLSYSNAME" true true false 28 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,TRLSYSNAME,0,27;GEN_TYPE "GEN_TYPE" true true false 10 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,GEN_TYPE,0,9;AGENCYNAME "AGENCYNAME" true true false 38 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,AGENCYNAME,0,37;AGCY_TYPE "AGCY_TYPE" true true false 16 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,AGCY_TYPE,0,15;SURFACETYP "SURFACETYP" true true false 18 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,SURFACETYP,0,17;PARL_RD "PARL_RD" true true false 30 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,PARL_RD,0,29;ALTPARLRD "ALTPARLRD" true true false 29 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,ALTPARLRD,0,28;YEAR_OPEN "YEAR_OPEN" true true false 4 Short 0 4,First,#,Hennepin_County_Bike_and_Pedestrian_System,YEAR_OPEN,-1,-1;SIDESTREET "SIDESTREET" true true false 10 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,SIDESTREET,0,9;RBTN "RBTN" true true false 1 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,RBTN,0,254;CLRZN_TYPE "CLRZN_TYPE" true true false 23 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,CLRZN_TYPE,0,22;CLRZN_WDTH "CLRZN_WDTH" true true false 24 Double 15 23,First,#,Hennepin_County_Bike_and_Pedestrian_System,CLRZN_WDTH,-1,-1;FACILITY "FACILITY" true true false 8 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,FACILITY,0,7;SPEC_TYPE "SPEC_TYPE" true true false 29 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,SPEC_TYPE,0,28;STATUS "STATUS" true true false 7 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,STATUS,0,6;COUNTYRD_R "COUNTYRD_R" true true false 3 Text 0 0,First,#,Hennepin_County_Bike_and_Pedestrian_System,COUNTYRD_R,0,2;ShapeSTLen "ShapeSTLen" true true false 24 Double 15 23,First,#,Hennepin_County_Bike_and_Pedestrian_System,ShapeSTLen,-1,-1',
    sort_field=None
)

#This is new basis for bike network - all other version will build from this

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Hennepin_Bike_Sys'>

In [11]:
'''
Will create three versions of bike network now:
1. Remove planned, but keep all bike facilities
2. Remove planned, keep only high quality bike facilities
3. Planned - all  bike facilities
'''
#Create 1 from above:
arcpy.conversion.ExportFeatures(
    in_features="Hennepin_Bike_Sys",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Hennepin_Current_Bike_Sys",
    where_clause="STATUS = 'Open'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='OBJECTID_1 "OBJECTID" true true false 2 Short 0 0,First,#,Hennepin_Bike_Sys,OBJECTID_1,-1,-1;CITY_MUNI "CITY_MUNI" true true false 33 Text 0 0,First,#,Hennepin_Bike_Sys,CITY_MUNI,0,32;COUNTY "COUNTY" true true false 8 Text 0 0,First,#,Hennepin_Bike_Sys,COUNTY,0,7;LENGTH "LENGTH" true true false 8 Double 0 0,First,#,Hennepin_Bike_Sys,LENGTH,-1,-1;WIDTH "WIDTH" true true false 8 Double 0 0,First,#,Hennepin_Bike_Sys,WIDTH,-1,-1;PARKING "PARKING" true true false 3 Text 0 0,First,#,Hennepin_Bike_Sys,PARKING,0,2;UNIQUE_ID "UNIQUE_ID" true true false 13 Text 0 0,First,#,Hennepin_Bike_Sys,UNIQUE_ID,0,12;TRAILNAME "TRAILNAME" true true false 53 Text 0 0,First,#,Hennepin_Bike_Sys,TRAILNAME,0,52;TRLSYSNAME "TRLSYSNAME" true true false 28 Text 0 0,First,#,Hennepin_Bike_Sys,TRLSYSNAME,0,27;GEN_TYPE "GEN_TYPE" true true false 10 Text 0 0,First,#,Hennepin_Bike_Sys,GEN_TYPE,0,9;AGENCYNAME "AGENCYNAME" true true false 38 Text 0 0,First,#,Hennepin_Bike_Sys,AGENCYNAME,0,37;AGCY_TYPE "AGCY_TYPE" true true false 16 Text 0 0,First,#,Hennepin_Bike_Sys,AGCY_TYPE,0,15;SURFACETYP "SURFACETYP" true true false 18 Text 0 0,First,#,Hennepin_Bike_Sys,SURFACETYP,0,17;PARL_RD "PARL_RD" true true false 30 Text 0 0,First,#,Hennepin_Bike_Sys,PARL_RD,0,29;ALTPARLRD "ALTPARLRD" true true false 29 Text 0 0,First,#,Hennepin_Bike_Sys,ALTPARLRD,0,28;YEAR_OPEN "YEAR_OPEN" true true false 2 Short 0 0,First,#,Hennepin_Bike_Sys,YEAR_OPEN,-1,-1;SIDESTREET "SIDESTREET" true true false 10 Text 0 0,First,#,Hennepin_Bike_Sys,SIDESTREET,0,9;RBTN "RBTN" true true false 1 Text 0 0,First,#,Hennepin_Bike_Sys,RBTN,0,254;CLRZN_TYPE "CLRZN_TYPE" true true false 23 Text 0 0,First,#,Hennepin_Bike_Sys,CLRZN_TYPE,0,22;CLRZN_WDTH "CLRZN_WDTH" true true false 8 Double 0 0,First,#,Hennepin_Bike_Sys,CLRZN_WDTH,-1,-1;FACILITY "FACILITY" true true false 8 Text 0 0,First,#,Hennepin_Bike_Sys,FACILITY,0,7;SPEC_TYPE "SPEC_TYPE" true true false 29 Text 0 0,First,#,Hennepin_Bike_Sys,SPEC_TYPE,0,28;STATUS "STATUS" true true false 7 Text 0 0,First,#,Hennepin_Bike_Sys,STATUS,0,6;COUNTYRD_R "COUNTYRD_R" true true false 3 Text 0 0,First,#,Hennepin_Bike_Sys,COUNTYRD_R,0,2;ShapeSTLen "ShapeSTLen" true true false 8 Double 0 0,First,#,Hennepin_Bike_Sys,ShapeSTLen,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,Hennepin_Bike_Sys,Shape_Length,-1,-1',
    sort_field=None
)



<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Hennepin_Current_Bike_Sys'>

In [12]:
arcpy.conversion.ExportFeatures(
    in_features="Hennepin_Current_Bike_Sys",
    out_features=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb\Henn_High_Qual_Bike_Sys",
    where_clause="SPEC_TYPE = 'Bike/Pedestrian Bridge' Or SPEC_TYPE = 'Bike/Pedestrian Tunnel' Or SPEC_TYPE = 'Bike-Only Trail' Or SPEC_TYPE = 'Contraflow Bike Lane' Or SPEC_TYPE = 'Cycle Track' Or SPEC_TYPE = 'Multi-Use Trail' Or SPEC_TYPE = 'On-Street Protected Bike Lane' Or SPEC_TYPE = 'Separated-Use Trail'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='OBJECTID_1 "OBJECTID" true true false 2 Short 0 0,First,#,Hennepin_Current_Bike_Sys,OBJECTID_1,-1,-1;CITY_MUNI "CITY_MUNI" true true false 33 Text 0 0,First,#,Hennepin_Current_Bike_Sys,CITY_MUNI,0,32;COUNTY "COUNTY" true true false 8 Text 0 0,First,#,Hennepin_Current_Bike_Sys,COUNTY,0,7;LENGTH "LENGTH" true true false 8 Double 0 0,First,#,Hennepin_Current_Bike_Sys,LENGTH,-1,-1;WIDTH "WIDTH" true true false 8 Double 0 0,First,#,Hennepin_Current_Bike_Sys,WIDTH,-1,-1;PARKING "PARKING" true true false 3 Text 0 0,First,#,Hennepin_Current_Bike_Sys,PARKING,0,2;UNIQUE_ID "UNIQUE_ID" true true false 13 Text 0 0,First,#,Hennepin_Current_Bike_Sys,UNIQUE_ID,0,12;TRAILNAME "TRAILNAME" true true false 53 Text 0 0,First,#,Hennepin_Current_Bike_Sys,TRAILNAME,0,52;TRLSYSNAME "TRLSYSNAME" true true false 28 Text 0 0,First,#,Hennepin_Current_Bike_Sys,TRLSYSNAME,0,27;GEN_TYPE "GEN_TYPE" true true false 10 Text 0 0,First,#,Hennepin_Current_Bike_Sys,GEN_TYPE,0,9;AGENCYNAME "AGENCYNAME" true true false 38 Text 0 0,First,#,Hennepin_Current_Bike_Sys,AGENCYNAME,0,37;AGCY_TYPE "AGCY_TYPE" true true false 16 Text 0 0,First,#,Hennepin_Current_Bike_Sys,AGCY_TYPE,0,15;SURFACETYP "SURFACETYP" true true false 18 Text 0 0,First,#,Hennepin_Current_Bike_Sys,SURFACETYP,0,17;PARL_RD "PARL_RD" true true false 30 Text 0 0,First,#,Hennepin_Current_Bike_Sys,PARL_RD,0,29;ALTPARLRD "ALTPARLRD" true true false 29 Text 0 0,First,#,Hennepin_Current_Bike_Sys,ALTPARLRD,0,28;YEAR_OPEN "YEAR_OPEN" true true false 2 Short 0 0,First,#,Hennepin_Current_Bike_Sys,YEAR_OPEN,-1,-1;SIDESTREET "SIDESTREET" true true false 10 Text 0 0,First,#,Hennepin_Current_Bike_Sys,SIDESTREET,0,9;RBTN "RBTN" true true false 1 Text 0 0,First,#,Hennepin_Current_Bike_Sys,RBTN,0,254;CLRZN_TYPE "CLRZN_TYPE" true true false 23 Text 0 0,First,#,Hennepin_Current_Bike_Sys,CLRZN_TYPE,0,22;CLRZN_WDTH "CLRZN_WDTH" true true false 8 Double 0 0,First,#,Hennepin_Current_Bike_Sys,CLRZN_WDTH,-1,-1;FACILITY "FACILITY" true true false 8 Text 0 0,First,#,Hennepin_Current_Bike_Sys,FACILITY,0,7;SPEC_TYPE "SPEC_TYPE" true true false 29 Text 0 0,First,#,Hennepin_Current_Bike_Sys,SPEC_TYPE,0,28;STATUS "STATUS" true true false 7 Text 0 0,First,#,Hennepin_Current_Bike_Sys,STATUS,0,6;COUNTYRD_R "COUNTYRD_R" true true false 3 Text 0 0,First,#,Hennepin_Current_Bike_Sys,COUNTYRD_R,0,2;ShapeSTLen "ShapeSTLen" true true false 8 Double 0 0,First,#,Hennepin_Current_Bike_Sys,ShapeSTLen,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,Hennepin_Current_Bike_Sys,Shape_Length,-1,-1',
    sort_field=None
)

<Result 'C:\\Users\\KOlso\\Documents\\ArcGIS\\Projects\\5571_Final_Proj\\5571_Final_Proj.gdb\\Henn_High_Qual_Bike_Sys'>

In [ ]:
#create feature dataset

arcpy.management.CreateFeatureDataset(
    out_dataset_path=r"C:\Users\KOlso\Documents\ArcGIS\Projects\5571_Final_Proj\5571_Final_Proj.gdb",
    out_name="Bike_analysis",
    spatial_reference='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision'
)

#need to move bike sysyem feature classes into here

In [ ]:
#Next steps are to use arcpy to initiate network dataset, set network properties, and conduct basics of network analysis to compare different networks